In [2]:
include("../Algebra/Gradients.jl")
include("../Algebra/Hamiltonians.jl")
include("../Algebra/Matrices.jl")
include("../Algebra/Propagators.jl")
include("../Amplitudes/Chebyshev.jl")
include("../Costs/Costs.jl")
include("../Gates/Xgate.jl")

U2 (generic function with 1 method)

In [3]:
using LinearAlgebra, Optim, Plots
using BenchmarkTools, Distributions

In [4]:
ω_c = 5
ω_q = 2
χ = 3e-3
N = 8
amp_c = 1
amp_q = 1

1

In [5]:
I_qubit = Matrix(I,2,2)
I_cavity = Matrix(I,N,N)
X_gate = kron(x_gate(N, Array[[3,4]]), I_qubit)

X_gate = kron(x_gate(N, Array[[3,4], [5,6], [7,8]]), I_qubit)

θ = π/2.5
RZ = kron(Rz(N, Array[[3,4]], θ/2), I_qubit)
RX = kron(Rx(N, Array[[3,4]], θ/2), I_qubit)
Udue = kron(U2(N, Array[[3,4]], [θ/2 θ/3 θ/4] ), I_qubit)

sx = kron(I_cavity, [0 1; 1 0])
sy = kron(I_cavity, [0 -1im; 1im 0])

ψ_initial = rand(N)
ψ_initial = kron(ψ_initial/norm(ψ_initial), [0,1])

16-element Vector{Float64}:
 0.0
 0.007453026772634228
 0.0
 0.5671885537897892
 0.0
 0.44023017967556405
 0.0
 0.10499409720737328
 0.0
 0.3690159510632338
 0.0
 0.48579255984472147
 0.0
 0.06146682731998171
 0.0
 0.3122016513342436

In [6]:
ψ_final_u = Udue * ψ_initial
ψ_final_x = RX * ψ_initial
ψ_final_x_gate = X_gate * ψ_initial

16-element Vector{Float64}:
 0.0
 0.007453026772634228
 0.0
 0.5671885537897892
 0.0
 0.10499409720737328
 0.0
 0.44023017967556405
 0.0
 0.48579255984472147
 0.0
 0.3690159510632338
 0.0
 0.3122016513342436
 0.0
 0.06146682731998171

In [7]:
# generating matrices
a,adag,sp,sm,sz = generate_matrices(N)

([0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0 1 … 0 0; 0 0 … 0 0; … ; 0 0 … 0 1; 0 0 … 0 0], [0 0 … 0 0; 1 0 … 0 0; … ; 0 0 … 0 0; 0 0 … 1 0], [1 0 … 0 0; 0 -1 … 0 0; … ; 0 0 … 1 0; 0 0 … 0 -1])

In [40]:
disp(α) = exp((α * adag - α' * a))
ECD(β) = disp(β / 2) * sp + disp(- β / 2) * sm
ROT(ϕ, θ) = exp(-1im * θ / 2 * ( sx * cos(ϕ) + sy * sin(ϕ) )) 

16×16 Matrix{Float64}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  

In [9]:
function cost(coefficients, unitary, N)

    L = length(coefficients)
    β_i = coefficients[1: Int(L/4)]
    β_r = coefficients[Int(L/4) + 1 : Int(L/2)]

    θs = coefficients[Int(L/2) + 1 : Int(3 * L/4)]
    ϕs = coefficients[Int(3 * L/4) + 1 : end]

    propagator = Matrix{ComplexF64}(I,N,N)

    for i in 1:length(β_i)
        propagator = propagator * ROT(ϕs[i], θs[i]) * ECD(β_i[i] + β_r[i] * 1im)
    end

    return 1 - norm(tr(propagator*unitary') / N)^2
    
end

function cost_ecd_rot_ecd(coefficients, unitary, N)

    L = length(coefficients)
    β_i = coefficients[1: Int(L/4)]
    β_r = coefficients[Int(L/4) + 1 : Int(L/2)]

    θs = coefficients[Int(L/2) + 1 : Int(3 * L/4)]
    ϕs = coefficients[Int(3 * L/4) + 1 : end]

    propagator = Matrix{ComplexF64}(I,N,N)

    for i in 1:length(β_i)
        propagator = propagator * ECD(β_i[i] + β_r[i] * 1im) * ROT(ϕs[i], θs[i]) * ECD(β_i[i] + β_r[i] * 1im)'
    end

    return 1 - norm(tr(propagator*unitary') / N)^2
    
end

function gradient(coefficients, unitary, N, h, cost_before_increment)

    L = length(coefficients)
    gradient = zeros(L)

    for i in 1:L
        coeffs_copy = copy(coefficients)
        coeffs_copy[i] = coefficients[i] + h
        gradient[i] = (cost(coeffs_copy, unitary, N) - cost_before_increment) / h
    end

    return gradient

end

function gate_composition(coefficients, N, l_coeff)

    L = length(coefficients)
    β_i = coefficients[1: Int(L/4)]
    β_r = coefficients[Int(L/4) + 1 : Int(L/2)]

    θs = coefficients[Int(L/2) + 1 : Int(3 * L/4)]
    ϕs = coefficients[Int(3 * L/4) + 1 : end]

    propagator = Matrix{ComplexF64}(I,N,N)

    for i in 1:length(β_i)
        propagator = propagator * ROT(ϕs[i], θs[i]) * ECD(β_i[i] + β_r[i] * 1im)
    end

    return propagator
    
end

function cost_state_prep(coefficients, N, ψ_n, ψ_m)

    L = length(coefficients)
    β_i = coefficients[1: Int(L/4)]
    β_r = coefficients[Int(L/4) + 1 : Int(L/2)]

    θs = coefficients[Int(L/2) + 1 : Int(3 * L/4)]
    ϕs = coefficients[Int(3 * L/4) + 1 : end]

    propagator = Matrix{ComplexF64}(I,N,N)

    for i in 1:length(β_i)
        propagator = propagator * ROT(ϕs[i], θs[i]) * ECD(β_i[i] + β_r[i] * 1im)
    end

    return 1 - norm(ψ_m' * propagator * ψ_n)^2
    
end


function gradient_state_prep(coefficients, N, h, cost_before_increment, ψ_n, ψ_m)

    L = length(coefficients)
    gradient = zeros(L)

    for i in 1:L
        coeffs_copy = copy(coefficients)
        coeffs_copy[i] = coefficients[i] + h
        gradient[i] = (cost_state_prep(coeffs_copy, N, ψ_n, ψ_m) - cost_before_increment) / h
    end

    return gradient

end

gradient_state_prep (generic function with 1 method)

In [10]:
f(x) = cost(x, RX, N * 2)
g(x) = cost_state_prep(x, N * 2, ψ_initial, ψ_final_x_gate)

g (generic function with 1 method)

In [42]:
costs = []
for depth in 40:8:40
    it_costs = []
    for iteration in 1:1
        println("blocks: ", depth," iterations :", iteration)
        l_coeff = depth * 4
        coefficients = zeros(l_coeff)
        opt_coefficients = [0.2500893876465987, 0.24772213175458602, -2.216980236737755, -0.22605456138931265, 0.5713324191103684, -0.47223890754460046, 0.17106471446861152, 0.08811951397913012, 0.4501927004803899, -0.9257430236964918, -0.25558551879559765, 0.09997017969959682, -0.12396628223204423, -0.3377828045444183, 0.31760702886740777, 0.659301590192822, 0.22982193941111242, 0.21233158284210046, -0.5612305910198453, 0.2051008925925025, 0.052418632295470426, -0.11755837533449529, -0.6579161837824333, 0.6714469369630117, 0.006471462738552314, -0.5741520396112727, 0.21313662344139656, 0.7567605165977416, 0.40883443305096, 0.3923030188502918, -1.1993635865955081, -1.1017601138910076, 0.3489144273862663, 0.3539602169294391, -1.0994208335153213, 0.1034958585167782, -0.6234139795132616, 0.5196380518067016, -0.47275758775751064, 0.33886874642306014, -0.00756374466839802, -0.4714567631718482, -0.04905826975450651, -0.381526493298742, 0.535810564331, 0.3907378055664532, -0.21271971479969243, -0.495203205121086, 0.060177433993175315, -0.5512415423096849, 0.9130916817890923, 0.1510457610918632, 0.12112216926344488, -0.24461234696448772, -0.7577170731136275, -1.009924910282126, 0.04192637434384756, 0.9285545430383368, 0.6353833215083303, -0.7289038131381561, -0.6585973799551663, -0.46455727003191144, -0.6398958836026388, -0.5247853072794502, 2.164384373961394, 0.03197216703848542, 2.1768882487832943, -2.9057307795490157, 0.2667687223070538, -2.5788631622347045, -1.845250779993158, -1.8590704850360174, 2.5576211174700534, -3.1074087103861414, -0.42188898754349125, 2.9162851312384865, -2.0031939267912726, -1.2948408748585236, 0.8909472411389935, -0.2244664340315024, -1.466209766964299, -1.3808511607296632, -3.5197110925840276, -1.423795343347976, 1.1175826270654081, 3.117077721044757, -0.9120695866740083, -3.197796455948583, 1.449943256608, -1.4802008321138975, 3.121825356795982, -2.009223807936916, -1.0793848195915419, 1.2903779901578778, 0.5537776991553981, -3.1278737881685617, -0.9255387659462153, 1.4844934034116912, 2.2194108330204356, -2.2080308454758617, 1.0433925715029868, -1.3953033927337726, -1.9048522203354048, 0.30086985454213705, 2.921716559333701, -1.8128839369721241, -2.031929887764076, 0.882125443935499, 2.4459842856866003, -3.5634218989841826, -1.4732555584354343, 3.018988494020728, 3.327132381723104, -1.1633286694952374, 2.8876347740783315, 2.0707687942277966, -3.9096728996327283, -1.2512963379726292, -0.03140288641279104, -2.6959450626715378, -0.423139186092174, 0.38251775209709266, 0.8038731335339105, 2.2692896433561405, -1.1819995585789678, 1.0190884576979866, -1.2056156203721584, -2.111902214107587]   
        coefficients[1:Int(l_coeff / 2)] = [opt_coefficients[1:Int(length(opt_coefficients)/2)]..., (rand(Int(l_coeff / 2 - length(opt_coefficients)/2))*2 .-1)...]
        coefficients[Int(l_coeff / 2) + 1:end] = [opt_coefficients[Int(length(opt_coefficients)/2)+1:end]..., zeros(Int(l_coeff / 2 - length(opt_coefficients)/2))...]
        res = Optim.optimize(f, coefficients , LBFGS(), Optim.Options(show_trace=true, show_every=10, iterations = 200))
        push!(it_costs,Optim.minimum(res))
        println(Optim.minimum(res))
        println(Optim.minimizer(res))
    end
    println(minimum(it_costs))
    push!(costs,minimum(it_costs))
end

blocks: 40 iterations :1
Iter     Function value   Gradient norm 


     0     9.942017e-01     3.195584e-02
 * time: 0.0003159046173095703


    10     6.210337e-01     2.655898e-01
 * time: 63.365991830825806


    20     1.148051e-01     1.554372e-01
 * time: 113.85313200950623


    30     3.682181e-02     6.208728e-02
 * time: 174.35538697242737


...

   170     1.584285e-02     2.567603e-03
 * time: 1152.7366127967834


   180     1.571186e-02     5.096326e-03
 * time: 1221.320702791214


   190     1.559498e-02     3.535631e-03
 * time: 1293.0019869804382


   200     1.542999e-02     3.746581e-03
 * time: 1361.888398885727


0.015429985890194509
[0.7039137998650173, -0.2116574631457655, -2.4772064190972856, -0.2513316289421471, 0.836261533910422, -0.31856952288438894, -0.13907524620657155, 0.30497518756875897, 0.6020592529950957, -0.9377737095738727, 0.0009127737345543903, 0.17524652537632154, 0.0495157888451895, -0.10609114331162385, 0.30648158473565607, 0.5370497007074168, -0.03958991389947197, -0.04876761792279877, 0.14498910048828278, -0.18249754480343897, 0.14223149702626323, -0.21006844902785785, -0.26612617056015864, 0.19760891027829727, 0.29038297305650806, -0.23345263917084022, 0.1888178559856714, 0.38912693853239627, 0.29516552616124253, 0.359690708649233, -1.5676559955524139, -1.8115485410717505, 0.2597405066722436, 0.19930343201999673, -0.25001248947377713, 0.04187372951382802, -0.008939151958867518, 0.18385586451282124, 0.3136690264642119, 0.563488229485811, 0.11897375133268893, -0.06311793317420807, -0.4410425615172428, -0.08699877702619846, -0.03500208371811297, -0.01968202176079141, 0.00369

# stoc gradient descent

In [16]:
costs_final = []
for depth in 32:8:64
    it_costs = []
    for iteration in 1:1
        l_coeff = depth * 4
        coefficients = zeros(l_coeff)
        coefficients[1:Int(l_coeff / 2)] = rand(Int(l_coeff / 2))*2 .-1
        coefficients[Int(l_coeff / 2) + 1:end] = rand(Int(l_coeff / 2)) * 2 * pi .- pi
        beta1 = 0.8
        beta2 = 0.999
        alpha = 0.01
        m = zeros(length(coefficients))
        v = zeros(length(coefficients))
        coeffs_stored = []
        costs = []
        cost_prev_it = cost(coefficients, RX, N*2)  #cost_state_prep(coefficients, N * 2, ψ_initial, ψ_final_x_gate) #

        for i in 1:1000
            grad = gradient(coefficients, RX, N * 2, 1e-8, cost_prev_it) #gradient_state_prep(coefficients, N*2, 1e-8, cost_prev_it, ψ_initial, ψ_final_x_gate) #
            for n in 1:length(coefficients)
                m[n] = beta1 * m[n] + ( 1 - beta1 ) * grad[n]
                v[n] = beta2 * v[n] + ( 1 - beta2 ) * grad[n]^2
                m_norm = m[n] / ( 1 - beta1^i )
                v_norm = v[n] / ( 1 - beta2^i )
                coefficients[n] = coefficients[n] - alpha * m_norm / (sqrt(v_norm) + 1e-8)
            end
            cost_prev_it = cost(coefficients, RX, N*2)# cost_state_prep(coefficients, N * 2, ψ_initial, ψ_final_x_gate) #
            push!(costs, cost_prev_it)

            if i % 50 == 0
                if norm(grad) < 1e-6
                    break
                end
                println(i)
                println(cost_prev_it)
            end
        end
        println("finished: d = ", depth)
        println(costs[end])
        push!(it_costs,costs[end])
        println(coefficients)
    end
    println(it_costs)
    println(minimum(it_costs))
    push!(costs_final, minimum(it_costs))
end
costs_final

InterruptException: InterruptException: